In [4]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)  # test spam
    do_nonlocal()
    print("After nonlocal assignment:", spam)  # nonlocal spam
    do_global()
    print("After global assignment:", spam)   # in aid of python visualizer
                                              # https://pythontutor.com/visualize.html#mode=edit

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


In [5]:
def a_class_under_this_func():
    class ClassUnderFunc:
        pass



In [1]:
class MyClass:
    """A simple example class"""
    i = 12345

    # When a class defines an __init__() method, class instantiation automatically invokes __init__() for the newly created class instance.
    
    def f1(self):
        return 'hello world'

*Attribute references* use the standard syntax used for all attribute references in Python: `obj.name`. Valid attribute names are all the names that were in the class’s namespace when the class object was created. So, if the class definition looked like this:



then `MyClass.i` and `MyClass.f` are valid attribute references, returning an integer and a function object, respectively. Class attributes can also be assigned to, so you can change the value of `MyClass.i` by assignment. `__doc__` is also a valid attribute, returning the docstring belonging to the class: `"A simple example class"`.

Class *instantiation* uses function notation. Just pretend that the class object is a parameterless function that returns a new instance of the class. For example (assuming the above class):


In [2]:
x = MyClass()

In [3]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

x = Complex(3.0, -4.5)
x.r, x.i

(3.0, -4.5)

*data attributes* correspond to “instance variables” in Smalltalk, and to “data members” in C++. Data attributes need not be declared; like local variables, they spring into existence when they are first assigned to. For example, if `x` is the instance of `MyClass` created above, the following piece of code will print the value `16`, without leaving a trace:

In [4]:
x.counter = 1
while x.counter < 10:
    x.counter = x.counter * 2
print(x.counter)
del x.counter

16


Valid method names of an instance object depend on its class. By definition, all attributes of a class that are function objects define corresponding methods of its instances. So in our example, `x.f` is a valid method reference, since `MyClass.f` is a function, but `x.i` is not, since `MyClass.i` is not. But `x.f` is not the same thing as `MyClass.f` — it is a *method object*, not a function object.

In [7]:
x = MyClass()
print(x.f1())
print(MyClass.f1(x))

hello world
hello world


Actually, you may have guessed the answer: the special thing about methods is that the instance object is passed as the first argument of the function. In our example, the call `x.f()` is exactly equivalent to `MyClass.f(x)`. In general, calling a method with a list of *n* arguments is equivalent to calling the corresponding function with an argument list that is created by inserting the method’s instance object before the first argument.

If the name denotes a valid class attribute that is a function object, a method object is created by **packing (pointers to) the instance object and the function object just found together in an abstract object: this is the method object**. When the method object is called with an argument list, a new argument list is constructed from the instance object and the argument list, and the function object is called with this new argument list.

#### Class and Instance Variables

Generally speaking, instance variables are for data unique to each instance and class variables are for attributes and methods shared by all instances of the class:

In [1]:
class Dog:

    kind = 'canine'         # class variable shared by all instances

    def __init__(self, name):
        self.name = name    # instance variable unique to each instance

In [5]:
d = Dog("Fido")
e = Dog("Buddy")
d.name

'Fido'

In [4]:
e.name

'Buddy'

In [7]:
class Dog:

    tricks = []             # mistaken use of a class variable
    # For example, the tricks list in the following code should not be used as a class variable because just a single list would be shared by all Dog instances:

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)

In [8]:
d = Dog('Fido')
e = Dog('Buddy')

In [9]:
d.add_trick('roll over')

In [10]:
e.add_trick('play dead')

In [13]:
d.tricks  # unexpectedly shared by all dogs

['roll over', 'play dead']

In [14]:
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)

There is no shorthand for referencing data attributes (or other methods!) from within methods. I find that this actually increases the readability of methods: there is no chance of confusing local variables and instance variables when glancing through a method.

Methods may call other methods by using method attributes of the `self` argument:

In [1]:
class Bag:
    def __init__(self):
        self.data = []

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)

Execution of a derived class definition proceeds the same as for a base class. When the class object is constructed, the base class is remembered. This is used for resolving attribute references: if a requested attribute is not found in the class, the search proceeds to look in the base class. This rule is applied recursively if the base class itself is derived from some other class.

There’s nothing special about instantiation of derived classes: `DerivedClassName()` creates a new instance of the class. Method references are resolved as follows: the corresponding class attribute is searched, descending down the chain of base classes if necessary, and the method reference is valid if this yields a function object.

- Use [`isinstance()`](https://docs.python.org/3/library/functions.html#isinstance) to check an instance’s type: `isinstance(obj, int)` will be `True` only if `obj.__class__` is [`int`](https://docs.python.org/3/library/functions.html#int) or some class derived from [`int`](https://docs.python.org/3/library/functions.html#int).
- Use [`issubclass()`](https://docs.python.org/3/library/functions.html#issubclass) to check class inheritance: `issubclass(bool, int)` is `True` since [`bool`](https://docs.python.org/3/library/functions.html#bool) is a subclass of [`int`](https://docs.python.org/3/library/functions.html#int). However, `issubclass(float, int)` is `False` since [`float`](https://docs.python.org/3/library/functions.html#float) is not a subclass of [`int`](https://docs.python.org/3/library/functions.html#int).

#### Multiple Inheritance

For most purposes, in the simplest cases, you can think of the search for attributes inherited from a parent class as depth-first, left-to-right, not searching twice in the same class where there is an overlap in the hierarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was searched for in Base2, and so on.

Since there is a valid use-case for class-private members (namely to avoid name clashes of names with names defined by subclasses), there is limited support for such a mechanism, called *name mangling*. Any identifier of the form `__spam` (at least two leading underscores, at most one trailing underscore) is textually replaced with `_classname__spam`, where `classname` is the current class name with leading underscore(s) stripped. This mangling is done without regard to the syntactic position of the identifier, as long as it occurs within the definition of a class.

Name mangling is helpful for letting subclasses override methods without breaking intraclass method calls. For example:

In [3]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method

class MappingSubclass(Mapping):
    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

​The above example would work even if `MappingSubclass` were to introduce a `__update` identifier since it is replaced with `_Mapping__update` in the `Mapping` class and `_MappingSubclass__update` in the `MappingSubclass` class respectively.

Sometimes it is useful to have a data type similar to the Pascal “record” or C “struct”, bundling together a few named data items. The idiomatic approach is to use [`dataclasses`](https://docs.python.org/3/library/dataclasses.html#module-dataclasses) for this purpose:

In [4]:
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    dept: str
    salary: int

In [6]:
john = Employee("John", "Computer Lab", 1000)
john.dept

'Computer Lab'

A piece of Python code that expects a particular abstract data type can often be passed a class that emulates the methods of that data type instead. For instance, if you have a function that formats some data from a file object, you can define a class with methods [`read()`](https://docs.python.org/3/library/io.html#io.TextIOBase.read) and [`readline()`](https://docs.python.org/3/library/io.html#io.TextIOBase.readline) that get the data from a string buffer instead, and pass it as an argument.

What you're describing is known as "**duck typing**" in Python. Duck typing is a concept in dynamic programming languages like Python, where the type or class of an object is determined by its behavior (i.e., the methods and attributes it has) rather than its explicit type. In other words, if an object walks like a duck and quacks like a duck, it's treated as a duck, regardless of its actual type.

In this example:

1. We define a `StringEmulator` class that emulates some methods of a file-like object, `read` and `readline`, to read data from a string.
2. We have a `process_file` function that expects a file-like object as an argument and reads data from it.
3. We create an instance of `StringEmulator`, `string_emulator`, and pass it as an argument to `process_file`.
4. The `process_file` function works with `string_emulator` as if it were a real file object because it implements the necessary methods (`read` and `readline`) that the function expects.

**This illustrates the flexibility of duck typing in Python, where you can pass different objects that behave similarly to what your function expects**, regardless of their specific class, as long as they have the required methods and attributes.

In [9]:
class StringEmulator:
    def __init__(self, data: str):
        self.data = data
        self.position = 0

    def read(self, size=None):
        if size is None:
            result = self.data[self.position:]
            self.position = len(self.data)
        else:
            result = self.data[self.position:self.position + size]
            self.position += size
        return result

    def readline(self):
        line = ""
        while self.position < len(self.data):
            char = self.data[self.position]
            self.position += 1
            if char == '\n':
                break
            line += char
        return line

# Function that expects a file-like object
def process_file(file_obj):
    """
    Process data from a file-like object.

    :param file_obj: An object that emulates a file with a `read()` method.
    :raises TypeError: If the provided object does not have a `read` method.
    """
    if not hasattr(file_obj, 'read') or not callable(file_obj.read):
        raise TypeError("Argument must implement a 'read()' method.")

    data = file_obj.read()
    print("Data read:", data)

# Using the StringEmulator as an argument to the function
string_data = "Hello, world!\nThis is a test string."
string_emulator = StringEmulator(string_data)
process_file(string_emulator)


Data read: Hello, world!
This is a test string.


In [5]:
for element in [1, 2, 3]:
    print(element)
for element in (1, 2, 3):
    print(element)
for key in {'one':1, 'two':2}:
    print(key)
for char in "123":
    print(char)
for line in open("myfile.txt"):
    print(line.strip(), end='\n')

1
2
3
1
2
3
one
two
1
2
3
Aha, you find me! Smart boy!
26
Aha, you find me! Smart boy!


In [9]:
s = 'abc'
it = iter(s)
it

In [10]:
next(it)

'a'

In [11]:
it

In [12]:
next(it)

'b'

In [13]:
it

In [14]:
next(it)

'c'

In [19]:
try:
    next(it)
except StopIteration as e:
    print("Error: StopIteration")


Error: StopIteration


Having seen the mechanics behind the iterator protocol, it is easy to add iterator behavior to your classes. Define an [`__iter__()`](https://docs.python.org/3/library/stdtypes.html#container.__iter__) method which returns an object with a [`__next__()`](https://docs.python.org/3/library/stdtypes.html#iterator.__next__) method. If the class defines `__next__()`, then `__iter__()` can just return `self`:

In [20]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        """Start from len(data) - 1"""
        if self.index == 0:
            raise StopIteration
        self.index -= 1
        return self.data[self.index]

In [21]:
data = range(1,11)
rev = Reverse(data)
for item in rev:
    print(item, end=" ")

10 9 8 7 6 5 4 3 2 1 

[Generators](https://docs.python.org/3/glossary.html#term-generator) are a simple and powerful tool for creating iterators. They are written like regular functions but use the [`yield`](https://docs.python.org/3/reference/simple_stmts.html#yield) statement whenever they want to return data. Each time [`next()`](https://docs.python.org/3/library/functions.html#next) is called on it, the generator resumes where it left off (it remembers all the data values and which statement was last executed). An example shows that generators can be trivially easy to create:

- Once the function yields, the function is paused and the control is transferred to the caller.
- When the function terminates, `StopIteration `is raised automatically on further calls.
- Local variables and their states are remembered between successive calls.
- The generator function contains one or more yield statements instead of a return statement.
- As the methods like `_next_() `and _`iter_()` are implemented automatically, we can iterate through the items using next().

In [22]:
# Python code to illustrate generator, yield() and next().
def generator():
	t = 1
	print ('First result is ',t)
	yield t

	t += 1
	print ('Second result is ',t)
	yield t

	t += 1
	print('Third result is ',t)
	yield t

call = generator()
next(call)
next(call)
next(call)


First result is  1
Second result is  2
Third result is  3


3

There are various other expressions that can be simply coded similar to list comprehensions but instead of brackets we use **parenthesis**. These expressions are designed for situations where the generator is used right away by an enclosing function. Generator expression allows creating a generator without a yield keyword. However, it doesn’t share the whole power of the generator created with a yield function. Example :  

In [23]:
# Python code to illustrate generator expression
generator = (num ** 2 for num in range(10))
for num in generator:
	print(num)


0
1
4
9
16
25
36
49
64
81


In [25]:
string = 'geek'
li = list(string[i] for i in range(len(string)-1, -1, -1))
print(li)


['k', 'e', 'e', 'g']


In [26]:
xvec = [10, 20, 30]
yvec = [7, 5, 3]
sum(x*y for x,y in zip(xvec, yvec))         # dot product

260

In [27]:
unique_words = set(word for line in page  for word in line.split())

NameError: name 'page' is not defined

In [35]:
class Student:
    def __init__(self, name, gpa):
        self.name = name
        self.gpa = gpa

# Create a list of student objects
graduates = [
    Student("Alice", 3.9),
    Student("Bob", 3.8),
    Student("Charlie", 4.0),
    Student("David", 3.95)
]

# Find the valedictorian based on GPA and name
# valedictorian = max((student.gpa, student.name) for student in graduates)
valedictorian = max(graduates, key = lambda student : student.gpa)

# Print the valedictorian's information
print("Valedictorian:", valedictorian.name, "with GPA:", valedictorian.gpa)


Valedictorian: Charlie with GPA: 4.0


In [32]:
valedictorian = max(graduates, key=lambda student: student.name)
print("Valedictorian in name order:", valedictorian.name, "with GPA:", valedictorian.gpa)

Valedictorian in name order: David with GPA: 3.95
